# QA con metadata filtering e ricerca di una label nei documenti

In [ ]:
!pip install pypdf
!pip install langchain
!pip install openai==0.28.1
!pip install chromadb
!pip install tiktoken
!pip install pinecone-client
!pip install python-dotenv

In [2]:
from google.colab import drive, files
import os
from dotenv import load_dotenv, find_dotenv
drive.mount("/content/drive/", force_remount=False)
%cd "/content/drive/My Drive/"
load_dotenv('./Lab_Python/.env')

Mounted at /content/drive/
/content/drive/My Drive


True

In [3]:
import pandas as pd

fine_tuning_dataset = pd.read_json("./Tesi/fine_tune_improved.json")
dataset_query = pd.read_json("./Tesi/prova_domande_fine_tuning.json")

In [4]:
query1 = fine_tuning_dataset.prompt[0]
print(query1)

Qual è il modo per partecipare all'Avviso MicroPrestito della Regione Puglia? in riferimento al documento: 'Scheda MicroPrestito della Regione Puglia - edizione 2021.pdf'


In [5]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


api langsmith per vedere le chiamate

In [6]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "ls__0554eeed658141b4a601195abd6c737d"

Carico i documenti ed estraggo il contenuto della pagina e i suoi metadata

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader

pdf_directory = "Tesi/Documenti"

pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

docs = []

for pdf_file in pdf_files:
  file_path = os.path.join(pdf_directory, pdf_file)

  loader = PyPDFLoader(file_path)
  loaded_docs = loader.load()

  for doc in loaded_docs:
      doc.metadata["source"] = pdf_file
      docs.append(doc)


In [ ]:
docs[0]

Document(page_content='  \n \n14 ottobre  2021 Pag 1 di 2 Centro Servizi Sistema Puglia  \nwww.sistema.puglia.it  \n \n \nScheda: Avviso MicroPrestito  della  Regione  Puglia  \n \nInformazioni  generali  Titolo  bando  MicroPrestito  della  Regione  Puglia  \nDescrizione  L’Avviso  è stato  adottato  per far fronte  alle esigenze  delle  imprese  derivanti  dalle  \nripercussioni  economiche  causate  dall’emergenza  epidemiologia  Covid -19. La \nmisura vuole garantire la continuità dell’operatività aziendale  e la salvaguardia \ndell’occupazione.  \nLe disposizioni  dell’Avviso  sono  conformi  all’approvazione  dell’Aiuto  di Stato  SA. \n57021 -Italia -Covid19 regime quadro del 21 maggio 2020 da parte della  \nCommissione  europea  ai sensi dell’art.  108 del TFUE e  s.m.i.  \nRisultati  attesi  La misura  sostiene  l’accesso  al credito  attraverso  interventi  di micro -finanza  \nche consistono  in mutui  concessi  a tasso zero  e senza  garanzie.  \nDestinatari  Il sistema  ec

In [ ]:
# from langchain.prompts import ChatPromptTemplate

# from langchain.chat_models import ChatOpenAI
# from langchain.document_transformers.openai_functions import create_metadata_tagger
# from langchain.schema import Document
# from typing import Literal

# from pydantic import BaseModel, Field

# prompt = ChatPromptTemplate.from_template(
#     """Calcola le occorrenze di ciascuna parola nel documento e dammi la parola che mi aiuti a identificare il documento:
# {input}
# """
# )
# class Properties(BaseModel):
#     label: str = Field(description="keyword non presente nel titolo e ripetuta nel documento")

# # Must be an OpenAI model that supports functions
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

# document_transformer = create_metadata_tagger(Properties, llm=llm, prompt = prompt)

# enhanced_documents = document_transformer.transform_documents(docs[:9])

In [ ]:
# from collections import Counter

# # Assume docs è una lista di documenti
# docs = [...]

# # Unisci tutti i documenti in un unico testo
# full_text = ' '.join(docs)

# # Tokenizza il testo in parole
# words = full_text.split()

# # Calcola le occorrenze di ciascuna parola
# word_counts = Counter(words)

# # Trova la parola chiave più comune
# most_common_word, occurrences = word_counts.most_common(1)[0]

# # Ora most_common_word contiene la parola chiave più comune e occurrences il numero di occorrenze
# print(f"Parola chiave più comune: {most_common_word}, Occorrenze: {occurrences}")


Uso una OpenAI function chiamata metadata_tagger che passandogli una classe pydantic con le proprietà label(il nome di quello che vogliamo trovare) : str(il tipo) = description(cosa deve cercare la funzione)  

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_transformers.openai_functions import create_metadata_tagger
from langchain.schema import Document
from typing import Literal

from pydantic import BaseModel, Field

class Properties(BaseModel):
    label: str = Field(description="parola più importante con più occurrenze")

# Must be an OpenAI model that supports functions
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

document_transformer = create_metadata_tagger(Properties, llm=llm)

enhanced_documents = document_transformer.transform_documents(docs)

In [ ]:
enhanced_documents

[Document(page_content='  \n \n14 ottobre  2021 Pag 1 di 2 Centro Servizi Sistema Puglia  \nwww.sistema.puglia.it  \n \n \nScheda: Avviso MicroPrestito  della  Regione  Puglia  \n \nInformazioni  generali  Titolo  bando  MicroPrestito  della  Regione  Puglia  \nDescrizione  L’Avviso  è stato  adottato  per far fronte  alle esigenze  delle  imprese  derivanti  dalle  \nripercussioni  economiche  causate  dall’emergenza  epidemiologia  Covid -19. La \nmisura vuole garantire la continuità dell’operatività aziendale  e la salvaguardia \ndell’occupazione.  \nLe disposizioni  dell’Avviso  sono  conformi  all’approvazione  dell’Aiuto  di Stato  SA. \n57021 -Italia -Covid19 regime quadro del 21 maggio 2020 da parte della  \nCommissione  europea  ai sensi dell’art.  108 del TFUE e  s.m.i.  \nRisultati  attesi  La misura  sostiene  l’accesso  al credito  attraverso  interventi  di micro -finanza  \nche consistono  in mutui  concessi  a tasso zero  e senza  garanzie.  \nDestinatari  Il sistema  e

In [ ]:
print(len(docs))

119


Splitto il testo delle pagine dei documenti salvando i metadata

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3796,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)


split_docs = []

for document in enhanced_documents:
  metadata = document.metadata
  page_content = document.page_content


  split_content = text_splitter.create_documents([page_content])


  for content_chunk in split_content:
    print(content_chunk)
    split_docs.append(Document(metadata = metadata, page_content = content_chunk.page_content))



page_content='14 ottobre  2021 Pag 1 di 2 Centro Servizi Sistema Puglia  \nwww.sistema.puglia.it  \n \n \nScheda: Avviso MicroPrestito  della  Regione  Puglia  \n \nInformazioni  generali  Titolo  bando  MicroPrestito  della  Regione  Puglia  \nDescrizione  L’Avviso  è stato  adottato  per far fronte  alle esigenze  delle  imprese  derivanti  dalle  \nripercussioni  economiche  causate  dall’emergenza  epidemiologia  Covid -19. La \nmisura vuole garantire la continuità dell’operatività aziendale  e la salvaguardia \ndell’occupazione.  \nLe disposizioni  dell’Avviso  sono  conformi  all’approvazione  dell’Aiuto  di Stato  SA. \n57021 -Italia -Covid19 regime quadro del 21 maggio 2020 da parte della  \nCommissione  europea  ai sensi dell’art.  108 del TFUE e  s.m.i.  \nRisultati  attesi  La misura  sostiene  l’accesso  al credito  attraverso  interventi  di micro -finanza  \nche consistono  in mutui  concessi  a tasso zero  e senza  garanzie.  \nDestinatari  Il sistema  economico  e produ

In [ ]:
import pandas as pd

df= pd.DataFrame(split_docs)

df.to_csv("df_with metadata_tag", index=False)
df

,0,1,2
0,"(page_content, 14 ottobre 2021 Pag 1 di 2 Centro Servizi Sistema Puglia \nwww.sistema.puglia.it \n \n \nScheda: Avviso MicroPrestito della Regione Puglia \n \nInformazioni generali Titolo bando MicroPrestito della Regione Puglia \nDescrizione L’Avviso è stato adottato per far fronte alle esigenze delle imprese derivanti dalle \nripercussioni economiche causate dall’emergenza epidemiologia Covid -19. La \nmisura vuole garantire la continuità dell’operatività aziendale e la salvaguardia \ndell’occupazione. \nLe disposizioni dell’Avviso sono conformi all’approvazione dell’Aiuto di Stato SA. \n57021 -Italia -Covid19 regime quadro del 21 maggio 2020 da parte della \nCommissione europea ai sensi dell’art. 108 del TFUE e s.m.i. \nRisultati attesi La misura sostiene l’accesso al credito attraverso interventi di micro -finanza \nche consistono in mutui concessi a tasso zero e senza garanzie. \nDestinatari Il sistema economico e produttivo pugliese. Chi può partecipare Beneficiari Le microimprese ed i lavoratori autonomi iscritti al registro delle imprese, con \nsede operativa in Puglia che abbiano subito perdite di fatturato o incrementi di \ncosti in conseguenza dell’epidemia Covid -19 e delle relative misure di \ncontenimento, avendo avuto nel periodo compreso tra il 1° gennaio 2021 e il 30 \nsettembre 2021 ricavi inferiori ai ricavi del periodo dal \n1° gennaio 2019 al 30 settembre 2019. \nLe microimprese devono essere costituite nelle seguenti forme giuridiche: ditta \nindividuale, società in nome collettivo, società in accomandita semplice, società \ncooperative, società a responsabilità limitata (anche in forma unipersonale e \nsemplificata). \nL’elenco dei codici attività ammissibili è riportato nell’Allegato 1 dell’Avviso. \nLe imprese, nell’esercizio 2019, non devono aver generato \nricavi /compensi/fatturato annuali superiori a 400.000,00 €. \nSe l’esercizio 2019 si è concluso con una perdita, quest’ultima non deve essere \nstata superiore al 20% del fatturato. Cosa \nfinanzi a Operazioni finanziabili Spese di funzionamento: costi derivanti dalla gestione dell’attività d’impresa \nsostenuti (pagati) in data successiva alla partecipazione all’avviso ed entro 12 mesi \ndalla concessione delle agevolazioni. Risorse economiche Fonte delle risorse POR Puglia 2014 -2020 \nFonte specifica Azioni 3.6 e 3.8 del POR Puglia 2014 -2020. \nFondo Microprestito della Regione Puglia 2014 -2020. \nA sportello / A \nscadenza Avviso a sportello operativo sino al 31/12/2021 e, comunque, sino ad \nesaurimento fondi \nContributo massimo \nper progetto Il finanziamento erogabile potrà essere compreso tra 5.000 € e 30.000 € in ragione \ndel fatturato conseguito nel 2019. I mutui avranno durata massima pari a 60 mesi \n(più preammortamento della durata di 12 mesi). Le imprese, in regola con la \nrestituzione del finanziamento, ottengono un contributo pari al 20% dell’importo \ndello stesso finanziamento sotto forma di abbuono delle ultime 12 rate. \nGli aiuti sono cumulabili con altri eventuali strumenti emergenziali di cui alla \nComunicazione della Commissione n. 1863/2020 e ss.mm.ii. \nRisorse totali stanziate 80.000.000 € Come partecipare \n \nModalità di \npartecipazione \n \n La domanda di partecipazione potrà essere presentata direttamente \ndall’impresa che chiede l’agevolazione, previa iscrizione al portale Sistema \nPuglia al seguente indirizzo: https://microcredito2021.sistema.puglia.it . \nLa procedura di partecipazione è interamente telematica. \nL’impresa dovrà essere in possesso di firma digitale ed essere titolare)","(metadata, {'label': 'Avviso MicroPrestito della Regione Puglia', 'source': 'Scheda MicroPrestito della Regione Puglia - edizione 2021.pdf', 'page': 0})","(type, Document)"
1,"(page_content, di conto corrente business anche non dedicato esclusivamente al progetto.)","(metadata, {'label': 'Avviso MicroPrestito della Regione Puglia', 'source': 'Scheda MicroPrestito della Regione Puglia - edizione 2021.pdf', 'page'

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
df.head(30)


,0,1,2
0,"(page_content, 14 ottobre 2021 Pag 1 di 2 Centro Servizi Sistema Puglia \nwww.sistema.puglia.it \n \n \nScheda: Avviso MicroPrestito della Regione Puglia \n \nInformazioni generali Titolo bando MicroPrestito della Regione Puglia \nDescrizione L’Avviso è stato adottato per far fronte alle esigenze delle imprese derivanti dalle \nripercussioni economiche causate dall’emergenza epidemiologia Covid -19. La \nmisura vuole garantire la continuità dell’operatività aziendale e la salvaguardia \ndell’occupazione. \nLe disposizioni dell’Avviso sono conformi all’approvazione dell’Aiuto di Stato SA. \n57021 -Italia -Covid19 regime quadro del 21 maggio 2020 da parte della \nCommissione europea ai sensi dell’art. 108 del TFUE e s.m.i. \nRisultati attesi La misura sostiene l’accesso al credito attraverso interventi di micro -finanza \nche consistono in mutui concessi a tasso zero e senza garanzie. \nDestinatari Il sistema economico e produttivo pugliese. Chi può partecipare Beneficiari Le microimprese ed i lavoratori autonomi iscritti al registro delle imprese, con \nsede operativa in Puglia che abbiano subito perdite di fatturato o incrementi di \ncosti in conseguenza dell’epidemia Covid -19 e delle relative misure di \ncontenimento, avendo avuto nel periodo compreso tra il 1° gennaio 2021 e il 30 \nsettembre 2021 ricavi inferiori ai ricavi del periodo dal \n1° gennaio 2019 al 30 settembre 2019. \nLe microimprese devono essere costituite nelle seguenti forme giuridiche: ditta \nindividuale, società in nome collettivo, società in accomandita semplice, società \ncooperative, società a responsabilità limitata (anche in forma unipersonale e \nsemplificata). \nL’elenco dei codici attività ammissibili è riportato nell’Allegato 1 dell’Avviso. \nLe imprese, nell’esercizio 2019, non devono aver generato \nricavi /compensi/fatturato annuali superiori a 400.000,00 €. \nSe l’esercizio 2019 si è concluso con una perdita, quest’ultima non deve essere \nstata superiore al 20% del fatturato. Cosa \nfinanzi a Operazioni finanziabili Spese di funzionamento: costi derivanti dalla gestione dell’attività d’impresa \nsostenuti (pagati) in data successiva alla partecipazione all’avviso ed entro 12 mesi \ndalla concessione delle agevolazioni. Risorse economiche Fonte delle risorse POR Puglia 2014 -2020 \nFonte specifica Azioni 3.6 e 3.8 del POR Puglia 2014 -2020. \nFondo Microprestito della Regione Puglia 2014 -2020. \nA sportello / A \nscadenza Avviso a sportello operativo sino al 31/12/2021 e, comunque, sino ad \nesaurimento fondi \nContributo massimo \nper progetto Il finanziamento erogabile potrà essere compreso tra 5.000 € e 30.000 € in ragione \ndel fatturato conseguito nel 2019. I mutui avranno durata massima pari a 60 mesi \n(più preammortamento della durata di 12 mesi). Le imprese, in regola con la \nrestituzione del finanziamento, ottengono un contributo pari al 20% dell’importo \ndello stesso finanziamento sotto forma di abbuono delle ultime 12 rate. \nGli aiuti sono cumulabili con altri eventuali strumenti emergenziali di cui alla \nComunicazione della Commissione n. 1863/2020 e ss.mm.ii. \nRisorse totali stanziate 80.000.000 € Come partecipare \n \nModalità di \npartecipazione \n \n La domanda di partecipazione potrà essere presentata direttamente \ndall’impresa che chiede l’agevolazione, previa iscrizione al portale Sistema \nPuglia al seguente indirizzo: https://microcredito2021.sistema.puglia.it . \nLa procedura di partecipazione è interamente telematica. \nL’impresa dovrà essere in possesso di firma digitale ed essere titolare)","(metadata, {'label': 'Avviso MicroPrestito della Regione Puglia', 'source': 'Scheda MicroPrestito della Regione Puglia - edizione 2021.pdf', 'page': 0})","(type, Document)"
1,"(page_content, di conto corrente business anche non dedicato esclusivamente al progetto.)","(metadata, {'label': 'Avviso MicroPrestito della Regione Puglia', 'source': 'Scheda MicroPrestito della Regione Puglia - edizione 2021.pdf', 'page'

In [ ]:
split_docs[3]

Document(page_content="Puglia sviluppo  \nUNIONE  EUROPEA  \nFONDO  EUROPEO  DI SVILUPPO  REGIONALE  REGIONE  PUGLIA  \nDIPARTIMENTO  SVILUPPO  ECONOMICO,  \nINNOVAZIONE,  ISTRUZIONE,  \nFORMAZIONE  E LAVORO  Ministero  Sviluppo  Economico  \n1  \n  \n \nNIDI  – Nuove  iniziative d’impresa  \nStrumento  di ingegneria  finanziaria  \n \nAvviso  a sportello  \nL’iniziativa  viene  attuata  da Puglia  Sviluppo  S.p.A. – società  interamente  \npartecipata  dalla Regione  Puglia  – con il sostegno  dell’Unione  Europea,  \nattraverso il Programma  Operativo FESR -FSE 2014 – 2020.  \nCosa  trovo  in questa  scheda?  \nQuesta  scheda  contiene  alcune  informazioni  su NIDI strumento  di politica  attiva  \ndel lavoro della Regione  Puglia  che sostiene  le scelte  di autoimpiego  ed \nautoimprenditorialità  da parte  delle donne,  dei disoccupati,  dei precari,  di chi sta \nper perdere il  posto  di lavoro,  dei giovani.  \nL’obiettivo  di questa  scheda  è fornire  un primo  orientamento 

Qui provo a cambiare a mano una label decisa da me

In [ ]:
for doc in split_docs:
  if doc.metadata['source'] == "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf":
    doc.metadata['label'] = 'agevolazione NIDI'
split_docs[6].metadata

{'label': 'agevolazione NIDI',
 'source': "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf",
 'page': 3}

In [ ]:
for doc in split_docs:
  if doc.metadata['source'] == "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf" and doc.metadata['page'] == 2  :
    doc.metadata['argomento'] = 'Spese'

Inserisco tutte le label in una lista che mi salverò sul disco

In [ ]:
list_label = []
for doc in split_docs:
  list_label.append(doc.metadata['label'])

list_label = set(list_label)
list_label

{'Appendice A1',
 'Attivi Materiali',
 'Avviso',
 'Avviso 4/2016',
 'Avviso 40/2017',
 'Avviso MicroPrestito della Regione Puglia',
 'Avviso OF 22',
 'Avviso n. 40/2017',
 'Avviso pubblico',
 'Certificazione ex novo',
 'Codici Ateco',
 'Contratti di Programma',
 'Contratto di Programma',
 'Diploma Professionale',
 'Diploma professionale',
 'Dotazione finanziaria',
 'GOL',
 'Misura',
 'Misure',
 'Obiettivi',
 'Organismi accreditati',
 'Organismi formativi',
 'PIA',
 'PIA TURISMO',
 'PICCOLE IMPRESE',
 'Pass Laureati',
 'Pass Laureati 2022',
 'Pass Laureati 2023',
 'Passage',
 'Percorsi IeFP',
 'Puglia',
 'Puglia Sviluppo',
 'Puglia sviluppo',
 'Reddito di Dignità pugliese',
 'Regione Puglia',
 'Responsabile di Azione',
 'Responsabile di Procedimento',
 'Risorse economiche',
 'Tecnonidi',
 'Titolo II capo 3',
 'Titolo II capo 6',
 'Titolo bando',
 'VALORE DELLA COMPONENTE PATRIMONIALE IMMOBILIARE DELL’ISEE, AL NETTO DELLE DETRAZIONI, NON SUPERIORE A € 30.000,00;',
 'Voci di Spesa',
 'acc

In [ ]:
with open("lista_label.txt", "w") as f:
  for elemento in list_label:
    f.write(str(elemento) + "\n")
f.close()

In [ ]:
split_docs[5].metadata

{'label': 'agevolazione NIDI',
 'source': "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf",
 'page': 2,
 'argomento': 'Spese'}

provo ad aggiungere un secondo metadato specifico per un chunk

In [ ]:
for doc in split_docs:
  if doc.metadata['source'] == "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf" and doc.metadata['page'] == 6  :
    doc.metadata['argomento'] = 'Ulteriori Informazioni'

In [ ]:
split_docs[9].metadata

{'label': 'agevolazione NIDI',
 'source': "Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf",
 'page': 6,
 'argomento': 'Ulteriori Informazioni'}

In [ ]:
print(f'Numero di chunks {len(split_docs)}')

Numero di chunks 127


###Creazione dei Vector Store

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [8]:
from langchain.vectorstores import Chroma
persist_directory = './Tesi/Documenti/docs/chroma/'

Creo un vector store con Chroma

In [ ]:
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [10]:
vectordb = Chroma(persist_directory = persist_directory, embedding_function= embedding)

In [13]:
print(vectordb._collection.count())

127


In [ ]:
# !rm -rf ./Tesi/Documenti/docs/chroma/ # remove old database files if any

In [12]:
def insert_or_fetch_embeddings(index_name, chunks):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()

    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name in pinecone.list_indexes():
        print(f'Index {index_name} esiste. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creazione di index {index_name} e embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')

    return vector_store

In [ ]:
vector = embedding.embed_query(split_docs[0].page_content)
print(len(vector))

1536


In [14]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Numero totale di Token: {total_tokens}')
    print(f'Costo degli Embedding $: {total_tokens / 1000 * 0.0001:.6f}')

print_embedding_cost(split_docs)

NameError: ignored

In [ ]:
df = pd.read_csv

In [15]:
import pinecone

pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [16]:
pinecone.list_indexes()

['embedding-bandi']

In [ ]:
# indexes = pinecone.list_indexes()
# for i in indexes:
#   pinecone.delete_index(i)
#   print('ok')

ok


Mentre qui creo un vectorstore di Pinecone

In [18]:
index_name = 'embedding-bandi'
vector_store = insert_or_fetch_embeddings(index_name, split_docs)

Index embedding-bandi esiste. Loading embeddings ... Ok


provo a fare una similarity search su Pinecone filtrando una label

In [20]:
query = fine_tuning_dataset.prompt[5]
results = vector_store.similarity_search(query, filter = {'label': 'Agevolazione'} )
print(results)

[Document(page_content="Puglia sviluppo  \nUNIONE  EUROPEA  \nFONDO  EUROPEO  DI SVILUPPO  REGIONALE  REGIONE  PUGLIA  \nDIPARTIMENTO  SVILUPPO  ECONOMICO,  \nINNOVAZIONE,  ISTRUZIONE,  \nFORMAZIONE  E LAVORO  Ministero  Sviluppo  Economico  \n1  \n  \n \nNIDI  – Nuove  iniziative d’impresa  \nStrumento  di ingegneria  finanziaria  \n \nAvviso  a sportello  \nL’iniziativa  viene  attuata  da Puglia  Sviluppo  S.p.A. – società  interamente  \npartecipata  dalla Regione  Puglia  – con il sostegno  dell’Unione  Europea,  \nattraverso il Programma  Operativo FESR -FSE 2014 – 2020.  \nCosa  trovo  in questa  scheda?  \nQuesta  scheda  contiene  alcune  informazioni  su NIDI strumento  di politica  attiva  \ndel lavoro della Regione  Puglia  che sostiene  le scelte  di autoimpiego  ed \nautoimprenditorialità  da parte  delle donne,  dei disoccupati,  dei precari,  di chi sta \nper perdere il  posto  di lavoro,  dei giovani.  \nL’obiettivo  di questa  scheda  è fornire  un primo  orientamento

Mastering Pinecone’s Advanced Filtering Techniques
Pinecone’s filtering query language takes a leaf out of MongoDB’s book, employing a similar structure of query and projection operators. This allows us to create complex metadata filters using logical operators like AND and OR. Here are some operators you can use:


```

$eq - Equal to (number, string, boolean)
$ne - Not equal to (number, string, boolean)
$gt - Greater than (number)
$gte - Greater than or equal to (number)
$lt - Less than (number)
$lte - Less than or equal to (number)
$in - In array (string or number)
$nin - Not in array (string or number)
```



In [ ]:
with open("lista_label.txt", "r") as f:
    # Creo una lista vuota
    list_label = []
    # Scorro le righe del file
    for riga in f:
        # Rimuovo il carattere di nuova riga e converto la riga in intero
        elemento = riga.strip()        # Aggiungo l'elemento alla lista
        list_label.append(elemento)
f.close()
# Stampo la lista caricata
print(list_label)

['spese', 'Avviso 4/2016', 'Organismi formativi', 'imprese', 'PICCOLE IMPRESE', 'Misure', 'Risorse economiche', 'Certificazione ex novo', 'PIA', 'Regione Puglia', 'investimenti', 'percorsi IeFP', 'opere murarie e assimilabili riferite a immobili dichiarati inagibili o inabitabili e di fatto non utilizzati', 'Titolo bando', 'Puglia sviluppo', 'passage', 'Avviso 40/2017', 'Misura', 'responsabile del procedimento', 'Avviso OF 22', 'accreditamento', 'Responsabile di Azione', 'novità', 'Organismi accreditati', 'Pass Laureati 2022', 'Attivi Materiali', 'GOL', 'percorsi formativi', 'Obiettivi', 'Diploma professionale', 'Titolo II capo 3', 'Avviso n. 40/2017', 'Tecnonidi', 'Puglia Sviluppo', 'acquisto', 'bilanci', 'impianti di produzione di idrogeno rinnovabile', 'Passage', 'Dotazione finanziaria', 'VALORE DELLA COMPONENTE PATRIMONIALE IMMOBILIARE DELL’ISEE, AL NETTO DELLE DETRAZIONI, NON SUPERIORE A € 30.000,00;', 'Avviso MicroPrestito della Regione Puglia', 'Contratto di Programma', 'agevola

Qui mi stampo tutte le label che ho

In [ ]:
print(list_label)

['spese', 'Avviso 4/2016', 'Organismi formativi', 'imprese', 'PICCOLE IMPRESE', 'Misure', 'Risorse economiche', 'Certificazione ex novo', 'PIA', 'Regione Puglia', 'investimenti', 'percorsi IeFP', 'opere murarie e assimilabili riferite a immobili dichiarati inagibili o inabitabili e di fatto non utilizzati', 'Titolo bando', 'Puglia sviluppo', 'passage', 'Avviso 40/2017', 'Misura', 'responsabile del procedimento', 'Avviso OF 22', 'accreditamento', 'Responsabile di Azione', 'novità', 'Organismi accreditati', 'Pass Laureati 2022', 'Attivi Materiali', 'GOL', 'percorsi formativi', 'Obiettivi', 'Diploma professionale', 'Titolo II capo 3', 'Avviso n. 40/2017', 'Tecnonidi', 'Puglia Sviluppo', 'acquisto', 'bilanci', 'impianti di produzione di idrogeno rinnovabile', 'Passage', 'Dotazione finanziaria', 'VALORE DELLA COMPONENTE PATRIMONIALE IMMOBILIARE DELL’ISEE, AL NETTO DELLE DETRAZIONI, NON SUPERIORE A € 30.000,00;', 'Avviso MicroPrestito della Regione Puglia', 'Contratto di Programma', 'agevola

###Label retrieval dalla query

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Classifica la seguente query per ottenere una label dalla lista:
{input}
"""
)


In [ ]:
fine_tuning_dataset.prompt[5]

"Quali sono le modalità per accedere alle agevolazioni del Fondo? in riferimento al documento: 'Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf'"

Qui creo una funzione che data la query trovi,sempre usando la functions create_metadata_tagger, una label da filtrare nel vectorstore.
Lo faccio mettendo nelle proprietà label : Literal[tutta la lista dei label], purtroppo faccio copia incolla dalla lista stampata sopra, devo trovare un modo per passargli un attributo dinamico

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.document_transformers.openai_functions import create_metadata_tagger
from langchain.schema import Document
from typing import List, Literal
from pydantic import BaseModel

class Properties(BaseModel):
    label: Literal['spese', 'Avviso 4/2016', 'Organismi formativi', 'imprese', 'PICCOLE IMPRESE', 'Misure', 'Risorse economiche', 'Certificazione ex novo', 'PIA', 'Regione Puglia', 'investimenti', 'percorsi IeFP', 'opere murarie e assimilabili riferite a immobili dichiarati inagibili o inabitabili e di fatto non utilizzati', 'Titolo bando', 'Puglia sviluppo', 'passage', 'Avviso 40/2017', 'Misura', 'responsabile del procedimento', 'Avviso OF 22', 'accreditamento', 'Responsabile di Azione', 'novità', 'Organismi accreditati', 'Pass Laureati 2022', 'Attivi Materiali', 'GOL', 'percorsi formativi', 'Obiettivi', 'Diploma professionale', 'Titolo II capo 3', 'Avviso n. 40/2017', 'Tecnonidi', 'Puglia Sviluppo', 'acquisto', 'bilanci', 'impianti di produzione di idrogeno rinnovabile', 'Passage', 'Dotazione finanziaria', 'VALORE DELLA COMPONENTE PATRIMONIALE IMMOBILIARE DELL’ISEE, AL NETTO DELLE DETRAZIONI, NON SUPERIORE A € 30.000,00;', 'Avviso MicroPrestito della Regione Puglia', 'Contratto di Programma', 'agevolazioni', 'organismi accreditati', 'agevolazione NIDI', 'agevolazione', 'risorse disponibili', 'Pass Laureati', 'PIA TURISMO', 'documentazione fiscale dell’esercizio 2019', 'Voci di Spesa', 'soggetti accreditati', 'Titolo II capo 6', 'investimenti e agevolazioni', 'Appendice A1', 'spese ammissibili', 'Puglia', 'Pass Laureati 2023', 'Avviso pubblico', 'domanda', 'Avviso', 'Responsabile di Procedimento', 'Reddito di Dignità pugliese', 'Contratti di Programma', 'numero dei percorsi assegnati', 'offerta formativa', 'Diploma Professionale', 'aiuto', 'misura Tecno Nidi', 'imprese di piccole dimensioni', 'documento', 'scheda', 'Percorsi IeFP', 'Codici Ateco']

def get_label_from_query(query, llm_model="gpt-3.5-turbo-0613"):
    llm = ChatOpenAI(temperature=0, model=llm_model)

    metadata_tagger = create_metadata_tagger(Properties, llm)

    query_document = Document(page_content=query)
    enhanced_query = metadata_tagger.transform_documents([query_document])

    label = enhanced_query[0].metadata["label"]

    return label

# query = "voglio informazioni per le Medie imprese"
# result_label = get_label_from_query(query)
# print(f"Label: {result_label}")


Label: imprese


Creo una funzione che trovi la risposta usando il filtro dinamico che va passato al retriever

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone

def get_answer_with_dynamic_filter(query, vector_store, label_to_search, llm_model='gpt-3.5-turbo', temperature=0.7):
    llm = ChatOpenAI(model=llm_model, temperature=temperature)

    dynamic_label = label_to_search

    search_kwargs = {"k": 3}

    vectorStore = Pinecone.from_existing_index(index_name, embedding)

    retriever = vectorStore.as_retriever(filter={'label': {'$eq': dynamic_label}}, search_kwargs=search_kwargs)

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(query)

    return answer

result_label = 'Agevolazioni'
query = fine_tuning_dataset.prompt[5]
result = get_answer_with_dynamic_filter(query, vector_store, result_label)
print(result)


Per accedere alle agevolazioni del Fondo, è necessario seguire le seguenti modalità:

1. Iscriversi gratuitamente sul portale www.sistema.puglia.it.
2. Compilare la domanda preliminare sul portale, fornendo informazioni sul progetto, sui soggetti proponenti, sugli investimenti e sulle spese previste.
3. Non è richiesto l'invio di documenti cartacei o l'uso della PEC nella fase preliminare.
4. Successivamente, è necessario partecipare a un colloquio di tutoraggio presso Puglia Sviluppo S.p.A., dove verranno valutate la fattibilità del progetto e assistiti nella definizione del programma di investimenti.
5. Al termine del colloquio, verrà rilasciata una copia del verbale e dell'istanza di agevolazione.
6. Puglia Sviluppo adotterà il provvedimento di concessione delle agevolazioni o di inammissibilità dopo l'iter istruttorio.
7. Le imprese ammesse saranno convocate presso la sede di Puglia Sviluppo per la sottoscrizione del contratto di agevolazione e per partecipare a una sessione di ass

#Retrieval con ChromaDB
provo a fare la similarity search filtrando adesso la label sul vectorstore di Chroma

In [58]:
question = fine_tuning_dataset.prompt[54]
result_label = get_label_from_query(question)
print(result_label)
docs_result = vectordb.similarity_search(question,k=3, filter ={"label" : result_label})
for d in docs_result:
    print(d.metadata)

spese
{'label': 'spese', 'page': 4, 'source': 'Aiuti agli Investimenti delle Piccole e Medie Imprese (Titolo II Capo 3 Reg_ Regionale 17_2014)_br ___br __.pdf'}
{'label': 'spese', 'page': 5, 'source': 'Aiuti Agli Investimenti delle PMI nel Settore Turistico - Alberghiero (Titolo II capo 6 Reg_ Reg_ 17_2014) _br ___br __.pdf'}
{'label': 'spese', 'page': 3, 'source': 'Aiuti ai programmi di internazionalizzazione delle Piccole e Medie Imprese_br ___br __.pdf'}


In [ ]:
question = fine_tuning_dataset.prompt[5]
result_label = get_label_from_query(question)
print(result_label)
docs_resul = vectordb.similarity_search(question,k=3, filter ={"label" : result_label})
for d in docs_resul:
    print(d.metadata)

scheda
{'label': 'scheda', 'page': 0, 'source': 'Programmi Integrati di Agevolazioni PIA Piccole Imprese (Art_ 27 Reg_ Regionale 17_2014 e s_m_i_).pdf'}
{'label': 'scheda', 'page': 0, 'source': 'Aiuti agli Investimenti delle Piccole e Medie Imprese (Titolo II Capo 3 Reg_ Regionale 17_2014)_br ___br __.pdf'}
{'label': 'scheda', 'page': 0, 'source': 'Aiuti Agli Investimenti delle PMI nel Settore Turistico - Alberghiero (Titolo II capo 6 Reg_ Reg_ 17_2014) _br ___br __.pdf'}


##Addressing Specificity: working with metadata using self-query retriever
SelfQueryRettriever, che usa una LLM per estrarre:
1. La stringa query da usare nel vector search
2. Un filtro metadata da passargli

I vectordb supportano i metadata filters

In [59]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [60]:
metadata_field_info = [
    AttributeInfo(
        name="label",
        description="keyword del documento",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="The info the chunk is from, should be one of the pdf in it",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page with info",
        type="integer",
    )
]

In [61]:
!pip install lark-parser


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.1 MB/s eta 0:00:00


ISSUE: Purtroppo ricevo errore con la SelfQueryRetriever

In [64]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever


document_content_description = "Bandi Regione Puglia"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectordb, document_content_description, metadata_field_info, verbose=True
)

ImportError: ignored

In [62]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import lark

def get_answer_with_dynamic_filter(query, vectordb, label_to_search, llm_model='gpt-3.5-turbo', temperature=0.7):
    llm = ChatOpenAI(model=llm_model, temperature=temperature)

    search_kwargs = {"k": 3}

    document_content_description = label_to_search
    retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
    )

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(query)

    return answer

result_label = get_label_from_query(query)
print(result_label)

query = fine_tuning_dataset.prompt[5]
result = get_answer_with_dynamic_filter(query, vectordb, result_label)
print(result)


imprese


ImportError: ignored

Dal retriever provo a trovare i documenti rilevanti sempre filtrandogli la label che trova dinamicamente

In [70]:
query = fine_tuning_dataset.prompt[55]
result_label = get_label_from_query(query)
print(result_label)
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents(query, filter = {'label' : result_label})

Avviso Tecnonidi


In [71]:
for d in docs:
    print(d.metadata)

{'label': 'agevolazioni', 'page': 2, 'source': 'Scheda Avviso Tecnonidi - Aiuti alle piccole imprese innovative.pdf'}
{'label': 'Tecnonidi', 'page': 0, 'source': 'Scheda Avviso Tecnonidi - Aiuti alle piccole imprese innovative.pdf'}
{'label': 'Voci di Spesa', 'page': 5, 'source': 'Programmi Integrati di Agevolazioni PIA Piccole Imprese (Art_ 27 Reg_ Regionale 17_2014 e s_m_i_).pdf'}
{'label': 'domanda', 'page': 8, 'source': 'Aiuti agli Investimenti delle Piccole e Medie Imprese (Titolo II Capo 3 Reg_ Regionale 17_2014)_br ___br __.pdf'}


La funzione di QA è identica a quella di Pinecone cambia soltanto il retriever

In [72]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma

def get_answer_with_dynamic_filter(query, vector_store, label_to_search, llm_model='gpt-3.5-turbo', temperature=0.7):
    llm = ChatOpenAI(model=llm_model, temperature=temperature)

    dynamic_label = label_to_search

    search_kwargs = {"k": 3}


    retriever = vector_store.as_retriever(filter={'label': {'$eq': dynamic_label}}, search_kwargs=search_kwargs)

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(query)

    return answer


query = fine_tuning_dataset.prompt[55]
print(query)
result_label = get_label_from_query(query)
result = get_answer_with_dynamic_filter(query, vectordb, result_label)
print(result)


Quali sono i requisiti necessari per poter beneficiare delle agevolazioni di Tecnonidi? in riferimento al documento: 'Scheda Avviso Tecnonidi - Aiuti alle piccole imprese innovative.pdf'
Per poter beneficiare delle agevolazioni di Tecnonidi, è necessario che l'impresa rientri in una delle seguenti tipologie:

a) Imprese regolarmente costituite ed iscritte al registro delle Start-up innovative;
b) Imprese che hanno sostenuto costi di ricerca e sviluppo che rappresentino almeno il 10% del totale dei costi di esercizio in almeno uno dei tre anni precedenti la concessione dell'aiuto oppure, nel caso di una start-up senza dati finanziari precedenti, nella revisione contabile dell'esercizio finanziario in corso;
c) Imprese che dispongono del riconoscimento del Seal of excellence.

Inoltre, le imprese devono essere di piccole dimensioni e devono avere intenzione di avviare o sviluppare piani di investimento a contenuto tecnologico nel territorio della Regione Puglia.

Si consiglia di consulta